In [21]:
import pandas as pd
import numpy as np
import random
import string
from faker import Faker
import faker_nonprofit

fake = Faker()
fake.add_provider(faker_nonprofit.Provider) 

In [22]:
#fake.name().split()[0]
#fake.pystr(min_chars=4, max_chars=10)
#[(fake.first_name(), fake.last_name()) for _ in range(10)]

In [23]:
def col_eg(df, col):
   return df[(~df[col].isna()) & (df[col]!="nan")][col][1:10]

In [24]:
client = "national_scleroderma_foundation"
path = "~/data"
filename =  "%s_constituent_ccsdb.csv" %(client)
file = "%s/%s" %(path, filename)
df_cd = pd.read_csv(file, encoding="ISO-8859-1")
df_cd = df_cd.dropna(how="all", axis=1)
df_cd = df_cd.rename(columns={"solicit_codes": "solicit_codes_binary"})
df_cd["solicit_codes_binary"] = df_cd["solicit_codes_binary"].astype("bool")
df_cd = df_cd.drop(columns=["Unnamed: 0"])

/var/folders/0m/d2nm9kxx5lxf971ys13kwry00000gn/T/ipykernel_52269/185851694.py:5: DtypeWarning: Columns (6,19,22,24,26,27,29,31,32,35,39,40,50) have mixed types. Specify dtype option on import or set low_memory=False.
  df_cd = pd.read_csv(file, encoding="ISO-8859-1")


In [25]:
df_cd_syn = df_cd.copy()
columns_nans ={key: df_cd_syn[key].isna().sum() for key in df_cd_syn.columns}

In [26]:
df_cd_syn[df_cd_syn.select_dtypes(include='object').columns] = \
                        df_cd_syn.select_dtypes(include='object')\
                                 .apply(lambda x: x.str.replace("scleroderma", "Company", case=False))

In [27]:
df_cd_syn["current_trustee"] = df_cd_syn["current_trustee"].replace("National Board of Directors", "Myown Board of Directors")
df_cd_syn["wealth_screen_data"] = df_cd_syn["wealth_screen_data"].replace("Blackbaud Analytics - NXT", "BB Analytics")

In [28]:
def create_array_with_nans(numbers, nans):
    # Combine the numbers and NaNs
    combined_list = numbers + [np.nan] * nans
    
    # Shuffle the combined list randomly
    np.random.shuffle(combined_list)
    
    # Convert the list to a numpy array
    result_array = np.array(combined_list)
    
    return result_array

# Example usage
# N1 = 5
# N2 = 3
# numbers_or_strings = [1, 2, "Alice", "Bob", 3.14]
# result = create_array_with_nans(numbers_or_strings, N2)

In [29]:
# Generate a list of 100 random strings
def generate_random_strings(num_strings, string_length):
    characters = string.ascii_letters + string.digits
    return [''.join(random.choice(characters) for _ in range(string_length)) for _ in range(num_strings)]

In [30]:
# Addresses

columns_address = ["address_1", "business_address", "seasonal_address"]

for col in columns_address:

    N = len(df_cd_syn) - columns_nans[col]

    # Generate a list of N random integers
    syn_house_numbers = [random.randint(1, N) for _ in range(N)]

    # Example: Generate a list of N random strings, each with a length of 10 characters
#    syn_street_names = generate_random_strings(num_strings=N, string_length=10)
    syn_street_names = [fake.street_name() for _ in range(N)]

    # List of street suffixes
    suffixes = ["Ave", "Str", "Rd"]
    syn_suffix = [random.choice(suffixes) for _ in range(N)]

    col_array = [f"{num} {name} {suffix}" for num, name, suffix \
                        in zip(syn_house_numbers, syn_street_names, syn_suffix)]
    df_cd_syn[col] = create_array_with_nans(col_array, columns_nans[col])
    df_cd_syn[col] = df_cd_syn[col].astype(str)

In [31]:
# Address_2

df_cd_syn["address_2"] = df_cd["address_2"].str.replace("apt", "Ste", case=False)
df_cd_syn["address_2"] = df_cd["address_2"].str.replace("ste", "Apt", case=False)
df_cd_syn["address_2"] = df_cd["address_2"].str.replace("dr", "Alley", case=False)

In [32]:
# Emails

columns_email = ["personal_email", "spouse_email"]

for col in columns_email:

    N = len(df_cd_syn) - columns_nans[col]

    # Example: Generate a list of N random strings, each with a length of 5 characters
    #syn_first_string = generate_random_strings(num_strings=N, string_length=5)
    syn_first_string = [fake.first_name() for _ in range(N)]

    # Example: Generate a list of N random strings, each with a length of 5 characters
    #syn_second_string = generate_random_strings(num_strings=N, string_length=5)
    syn_second_string = [fake.last_name() for _ in range(N)]

    # List of street suffixes
    suffixes = ["@gmail.com", "@yahoo.com", "@verizon.net", "@outlook.com"]
    syn_suffix = [random.choice(suffixes) for _ in range(N)]

    col_array = [f"{first}.{second}{suffix}" for first, second, suffix \
                        in zip(syn_first_string, syn_second_string, syn_suffix)]
    df_cd_syn[col] = create_array_with_nans(col_array, columns_nans[col])
    df_cd_syn[col] = df_cd_syn[col].astype(str)

In [33]:
# Other Fields

columns_to_synthesize_string = ["first_name", "middle_name", "last_name", "spouse_name", "spouse_business_name"]
for col in columns_to_synthesize_string:

    N = len(df_cd_syn) - columns_nans[col]

    if col=="first_name":
        col_array = [fake.first_name() for _ in range(N)]
    if col=="last_name":
        col_array = [fake.last_name() for _ in range(N)]
    if col=="middle_name":
        col_array = ["%s." %random.choice(string.ascii_letters) for _ in range(N)]
    if col=="spouse_name":
        col_array = [fake.name() for _ in range(N)]
    if col=="spouse_business_name":
        col_array = [fake.job() for _ in range(N)]
    
    df_cd_syn[col] = create_array_with_nans(col_array, columns_nans[col])
    df_cd_syn[col] = df_cd_syn[col].astype(str)

columns_to_synthesize_integer = ["constituent_id", "spouse_id", "home_phone", "cell_phone", "business_phone"]
for col in columns_to_synthesize_integer:

    N = len(df_cd) - columns_nans[col]
    
    if "phone" in col:
        col_array = [random.randint(1000000000, 9999999999) for _ in range(N)]
    else:
        col_array = [random.randint(1, N) for _ in range(N)]
    
    df_cd_syn[col] = create_array_with_nans(col_array, columns_nans[col])
    df_cd_syn[col] = df_cd_syn[col].astype(str)

#[(col, col_eg(df_cd_syn, col)) for col in columns_to_synthesize_string]

In [34]:
# Other Fields

columns_to_synthesize_string = ["constituent_type_1", "constituent_type_2", "age", "org_name", "zip"]
for col in columns_to_synthesize_string:

    N = len(df_cd_syn) - columns_nans[col]

    if col=="constituent_type_1":
        col_array = [fake.company() for _ in range(N)]
    if col=="constituent_type_2":
        col_array = [fake.job() for _ in range(N)]
    if col=="age":
        col_array = [fake.passport_dob() for _ in range(N)]
    if col=="org_name":
        col_array = [fake.company() for _ in range(N)]
    if col=="zip":
        col_array = [fake.postcode() for _ in range(N)]
    
    df_cd_syn[col] = create_array_with_nans(col_array, columns_nans[col])
    df_cd_syn[col] = df_cd_syn[col].astype(str)

In [35]:
col = "assigned_manager"

# Create Fake Assigned Managers
N_assigned_managers = df_cd_syn[col].nunique()

# List of fake assigned managers
fake_assigned_managers_list = [fake.name() for i in range(N_assigned_managers)]

N = len(df_cd_syn) - columns_nans[col]
col_array = [random.choice(fake_assigned_managers_list) for _ in range(N)]
df_cd_syn[col] = create_array_with_nans(col_array, columns_nans[col])
df_cd_syn[col] = df_cd_syn[col].astype(str)

In [40]:
df_cd_syn = df_cd_syn.rename(columns={"age": "dob", "spouse_age": "spouse_dob"})

In [44]:
df_cd_syn.columns

Index(['constituent_id', 'first_name', 'middle_name', 'last_name', 'address_1',
       'address_2', 'home_city', 'home_state', 'zip', 'home_country',
       'modification_date', 'constituent_type_1', 'constituent_type_2',
       'deceased', 'key_indicator', 'prefix', 'suffix', 'org_name',
       'spouse_id', 'spouse_name', 'spouse_dob', 'spouse_email',
       'spouse_business_name', 'spouse_business_title', 'business_address',
       'seasonal_address', 'home_phone', 'cell_phone', 'business_phone',
       'personal_email', 'marital_status', 'dob', 'wealth_screen_data',
       'current_trustee', 'past_trustee', 'history_of_volunteer',
       'number_of_special_events_attended', 'last_action_type',
       'last_action_date', 'date_of_last_note', 'total_notes',
       'assigned_manager', 'solicit_codes_binary', 'lifetime_giving',
       'five_year_giving', 'lifetime_hard_credits', 'first_gift_amount',
       'first_gift_date', 'most_recent_gift_amount', 'most_recent_gift_date',
       'nu

In [45]:
df_cd_syn.to_csv("~/data/synthetic_constituent_data.csv")